In [1]:
import akshare as ak
import pandas as pd
import numpy as np

# 设置绘图
from matplotlib import pyplot as plt
import seaborn as sns
sns.set()
#设置字体为SimHei显示中文
plt.rcParams['font.sans-serif'] = 'SimHei'
#设置正常显示字符
plt.rcParams['axes.unicode_minus'] = False
import requests,json


1. 爬取黄金交易所数据
2. 爬取黄金期货数据
3. 整理


In [2]:
# 现货实时行情

def au_real_time():
    url='https://www.sge.com.cn/graph/quotations'

    form_data={'instid':'Au(T+D)'}

    res=requests.post(url,data=form_data)
    import json
    au_td=pd.DataFrame(json.loads(res.text))[['times','data']].set_index('times')

    au_td
#     au_td.drop_duplicates(inplace=True,keep=False) # 删掉了一些有用的信息
    au_td=au_td.astype(float)
    au_td.replace(au_td.iloc[-1],np.nan,inplace=True)
    au_td.dropna(inplace=True)
    return au_td

au_td=au_real_time()
au_td

,data
times,
20:00,390.44
20:01,390.30
20:02,390.52
20:03,390.32
20:04,390.50
...,...
12:06,391.35
12:07,391.35
12:08,391.35


In [3]:
## 黄金期货实时行情
## 不行 有一个rn参数
# url='http://hq.sinajs.cn/rn=9oqyb&list=nf_AU0,nf_AU2101,nf_AU2102,nf_AU2103,nf_AU2104,nf_AU2106,nf_AU2108,nf_AU2110,nf_AU2112'

# real_time_au_res=requests.get(url)
# real_time_au=pd.DataFrame(list(map(lambda x:x.split(','),real_time_au_res.text.split('\n')))).iloc[:-1]
# real_time_au.replace('',np.nan,inplace=True)
# real_time_au.dropna(axis=1,inplace=True)
# real_time_au

In [4]:
# headers={'Accept':'*/*',
# 'Accept-Encoding':'gzip, deflate, br',
# 'Accept-Language':'zh-CN,zh;q=0.9,en;q=0.8,en-GB;q=0.7,en-US;q=0.6',
# 'Connection':'keep-alive',
# 'Host':'hq.sinajs.cn',
# 'Referer':'https://finance.sina.com.cn/futures/quotes/AU2106.shtml',
# 'Sec-Fetch-Dest':'script',
# 'Sec-Fetch-Mode':'no-cors',
# 'Sec-Fetch-Site':'cross-site',
# 'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36 Edg/87.0.664.66'}

In [5]:
# 有问题，sina这个反扒厉害了,好像其实没问题。。。
# import time
# print(time.time())
# url='https://hq.sinajs.cn/?_={}/&list=nf_AU2106,nf_AU2104,nf_AU2108,nf_AU2110,nf_AU2101,nf_AU2006'
# full_url=url.format(str(time.time())[:14].replace('.',''))
# print(full_url)
# res=requests.get(full_url,headers=headers)

# real_time_au=pd.DataFrame(list(map(lambda x:x.split(','),res.text.split('\n')))).iloc[:-1]
# real_time_au

In [6]:
# # 期货实时行情
# import time

# shfe_text = ak.match_main_contract(exchange="shfe")
# while True:
# #     time.sleep(3)
#     data = ak.futures_zh_spot(
#         subscribe_list=",".join([ shfe_text]),
#         market="CF")
#     au_data=data[data['symbol'].apply(lambda x:True if '黄金' in x else False)]
#     print(au_data['current_price'])
#     break

In [10]:
# 期货历史行情
def au_hist(date='20201228'):
    shfe_headers = {"User-Agent": "Mozilla/4.0 (compatible; MSIE 5.5; Windows NT)"}
    au_hist_url='http://www.shfe.com.cn/data/dailydata/kx/kx{}.dat'.format(date)
    au_history_json=json.loads(
        requests.get(
            au_hist_url,headers=shfe_headers).text
    )
    df = pd.DataFrame(
        [
            row
            for row in au_history_json["o_curinstrument"]
            if row["DELIVERYMONTH"] not in ["小计", "合计"] and row["DELIVERYMONTH"] != ""
        ]
    )
    df["variety"] = df.PRODUCTID.str.slice(0, -6).str.upper()
    df["symbol"] = df["variety"] + df["DELIVERYMONTH"]
    # df["date"] = day.strftime("%Y%m%d")
    au_history=df[df['PRODUCTID']=='au_f    ']

    
    SHFE_COLUMNS = {
        'symbol':'symbol',

        "CLOSEPRICE": "close",
        "HIGHESTPRICE": "high",
        "LOWESTPRICE": "low",
        "OPENINTEREST": "open_interest",
        "OPENPRICE": "open",
        "PRESETTLEMENTPRICE": "pre_settle",
        "SETTLEMENTPRICE": "settle",
        "VOLUME": "volume"
    }

    au_history=au_history[SHFE_COLUMNS.keys()]
    au_history.rename(columns=SHFE_COLUMNS,inplace=True)
    return au_history
au_hist(date='20201228')

,symbol,close,high,low,open_interest,open,pre_settle,settle,volume
82,AU2101,390.72,395.6,390.72,65,392,391.34,394.5,143
83,AU2102,395.78,399.2,387.96,43411,395,393.22,396.36,96263
84,AU2103,399.18,399.18,394.86,25,394.86,394.7,396.56,4
85,AU2104,396.82,399.88,393.12,35576,395.72,393.98,397.2,21500
86,AU2106,398.2,401.76,390,88335,396.7,396.14,399.16,51414
87,AU2108,399.62,403.44,396.78,21156,399.3,397.68,400.82,6964
88,AU2110,401.6,405.18,398.52,384,401.24,400.82,402.32,941
89,AU2112,401.06,406.66,401.02,108,402.88,401.74,403.52,21


In [18]:
def one_future_real_time(symbol='AU2106'):
    # 改为t5可获取5日的
    url='https://stock2.finance.sina.com.cn/futures/api/jsonp.php/var%20t1nf_AU2106=/InnerFuturesNewService.getMinLine?symbol={}'.format(symbol)
    res=requests.get(url)
    df=pd.DataFrame (eval(res.text[res.text.index('('):][1:-2]),columns=['times','price','avg_price','deal_amount','open_interest','info','info']).set_index('times').iloc[:,:-2]
    df.price=df.price.astype('float')

    return pd.DataFrame(df.price)
au_oneday_real_time=one_future_real_time()
au_oneday_real_time.tail()

,price
times,
11:26,397.10
11:27,397.06
11:28,397.04
11:29,396.88
11:30,396.94


In [35]:
from collections import Counter #引入Counter
a=au_oneday_real_time.index.tolist()
b = dict(Counter(a))
print ([key for key,value in b.items()if value > 1]) #只展示重复元素
print ({key:value for key,value in b.items()if value > 1}) #展现重复元素和重复次数

['11:30']
{'11:30': 2}


In [114]:
future_oneday_real_time=one_future_real_time()
au_oneday_real_time=au_real_time()
au_oneday_real_time=au_oneday_real_time[~au_oneday_real_time.index.duplicated(keep='first')] # 因为可能有重复值，去重
au_and_future=pd.concat([future_oneday_real_time,au_oneday_real_time],axis=1)
au_and_future.columns=['future','Au_TD']

au_and_future

,future,Au_TD
21:00,396.98,391.40
21:01,396.86,391.24
21:02,396.88,391.23
21:03,397.00,391.30
21:04,397.24,391.58
...,...,...
15:02,NaN,390.27
15:03,NaN,390.27
15:04,NaN,390.20
15:05,NaN,390.20


In [122]:

au_and_future['diff'] = au_and_future['future'] - au_and_future['Au_TD']

delivery_day = datetime.datetime.strptime('2021-6-16', '%Y-%m-%d') #到期日
today = datetime.datetime.today()
day_to_delivery = (delivery_day - today).days + 1  # 补上半天的差
au_and_future['ytm']=au_and_future['diff'] * 365 / ((day_to_delivery) * au_and_future['Au_TD'])*100
au_and_future

,future,Au_TD,diff,ytm
21:00,396.98,391.40,5.58,3.079070
21:01,396.86,391.24,5.62,3.102410
21:02,396.88,391.23,5.65,3.119051
21:03,397.00,391.30,5.70,3.146090
21:04,397.24,391.58,5.66,3.121779
...,...,...,...,...
15:02,NaN,390.27,NaN,NaN
15:03,NaN,390.27,NaN,NaN
15:04,NaN,390.20,NaN,NaN
15:05,NaN,390.20,NaN,NaN


In [65]:
# 黄金期货高频实时
import time
def future_real_time(contract_list='nf_AU0,nf_AU2101,nf_AU2102,nf_AU2103,nf_AU2104,nf_AU2106,nf_AU2108,nf_AU2110,nf_AU2112'.split(',')):
    url = f"https://hq.sinajs.cn/rn={round(time.time()*1000)}&list={','.join(contract_list)}"
    res = requests.get(url)
    data_df = pd.DataFrame([item.strip().split("=")[1].split(
        ",") for item in res.text.split(";") if item.strip() != ""]) # 获取等号后的值
    data_df.iloc[:, 0] = data_df.iloc[:, 0].str.replace('"', "") # 第一列处理
    data_df.iloc[:, -1] = data_df.iloc[:, -1].str.replace('"', "")
    data_df.replace('',np.nan,inplace=True)
    data_df.dropna(axis=1,inplace=True)
    data_df.columns=[['name','info','open','high','low','settle','buy','sale','current_price',
                      'info2','pre_settle','buy_amount','sale_amount','open_interest','deal_amount',
                      ]+['info']*5]
    print(data_df)
    return data_df[['name','buy','sale','current_price',
                      'buy_amount','sale_amount','open_interest','deal_amount',
                      ]]
future_real_time(contract_list='nf_AU0,nf_AU2101,nf_AU2102,nf_AU2103,nf_AU2104,nf_AU2106,nf_AU2108,nf_AU2110,nf_AU2112'.split(','))

     name    info     open     high      low settle      buy     sale  \
0    黄金连续  143918  394.160  395.280  392.140  0.000  393.540  393.560   
1  黄金2101  143425  391.200  395.960  391.000  0.000  392.040  393.580   
2  黄金2102  143918  394.160  395.280  392.140  0.000  393.540  393.560   
3  黄金2103  143819  394.580  395.500  394.060  0.000  393.880  394.820   
4  黄金2104  143918  394.580  395.760  392.740  0.000  394.200  394.260   
5  黄金2106  143915  395.740  397.540  394.400  0.000  395.900  395.920   
6  黄金2108  143917  397.980  399.400  396.460  0.000  397.640  397.720   
7  黄金2110  143917  399.880  401.080  398.480  0.000  399.580  399.660   
8  黄金2112  143911  401.420  402.560  401.040  0.000  401.060  401.900   

  current_price  info2 pre_settle buy_amount sale_amount open_interest  \
0       393.540  0.000    395.620          5          13     39735.000   
1       392.940  0.000    392.820          3           1        21.000   
2       393.540  0.000    395.620          5   

,name,buy,sale,current_price,buy_amount,sale_amount,open_interest,deal_amount
0,黄金连续,393.540,393.560,393.540,5,13,39735.000,108812
1,黄金2101,392.040,393.580,392.940,3,1,21.000,40
2,黄金2102,393.540,393.560,393.540,5,13,39735.000,108812
3,黄金2103,393.880,394.820,395.500,1,1,28.000,4
4,黄金2104,394.200,394.260,394.240,2,3,37569.000,26262
5,黄金2106,395.900,395.920,395.900,5,3,88269.000,40549
6,黄金2108,397.640,397.720,397.680,2,5,21460.000,7957
7,黄金2110,399.580,399.660,399.480,8,4,1614.000,683
8,黄金2112,401.060,401.900,401.860,1,1,114.000,12


In [172]:
future_info=future_real_time(contract_list='nf_AU2106'.split(',')).iloc[0]
future_info

name                黄金2106
buy                396.460
sale               396.500
current_price      396.480
buy_amount               4
sale_amount              1
open_interest    88614.000
deal_amount          12273
Name: 0, dtype: object

In [ ]:
"""
     /**
      * 渲染头部
      * @param {object} items 
      * zxj: 最新价
      * zde: 涨跌额
      * zdf: 涨跌幅
      * jkj: 今开价
      * zjs: 昨结算
      * zgj: 最高价
      * zdj：最低价
      * cjl: 成交量
      * ccl: 持仓量
      * wp:外盘
      * np:内盘
      * cc:仓差
      * rz:日增
      * zsj：昨收价
      * mcj:卖出价
      * mcl:卖出量
      * mrj:买入价
      * mrl:买入量
      * dtj：跌停价
      * cje：成交额
      * kp： 开平
      * quote_title_0：名称
      * quote_title_1： 代码
      * jzjs： 今结算价
      * zsjd：展示精度
      */
      """

In [48]:
from selenium import webdriver
driver=webdriver.Chrome()
east_url='http://quote.eastmoney.com/globalfuture/AUTD.html'
driver.get(east_url)

In [49]:
from bs4 import BeautifulSoup

import re
text = driver.page_source
soup=BeautifulSoup(text,'html5lib')
buy_sale_price=soup.find_all('table',class_='maimai')[0].text # 买卖信息
buy_sale_price=re.split('[\n ]+',buy_sale_price.strip())
current_price=soup.find_all('i',class_='zxj')[0].text # 买卖信息


buy_sale_price,current_price

(['卖一', '390.24', '1', '买一', '390.14', '1'], '390.13')

In [50]:
driver.quit()

In [182]:



au_and_future['diff'] = au_and_future['future'] - au_and_future['Au_TD']

delivery_day = datetime.datetime.strptime('2021-6-16', '%Y-%m-%d') #到期日
today = datetime.datetime.today()
day_to_delivery = (delivery_day - today).days + 1  # 补上半天的差
au_and_future['ytm']=au_and_future['diff'] * 365 / ((day_to_delivery) * au_and_future['Au_TD'])*100



price_dict={'xh_sale_p':buy_sale_price[1],'xh_buy_p':buy_sale_price[4],'xh_now_p':current_price,
                'qh_sale_p':future_info[2],'qh_buy_p':future_info[1],'qh_now_p':future_info[3]}

def get_ytm(buy,sale,now,date='2021-6-16'):
    delivery_day = datetime.datetime.strptime(date, '%Y-%m-%d') #到期日
    today = datetime.datetime.today()
    day_to_delivery = (delivery_day - today).days + 1  # 补上半天的差

    ytm=(buy-sale)* 365 / ((day_to_delivery) * now)*100
    return ytm

for i,k in price_dict.items():
    price_dict[i]=float(k)
price_dict['zhengtao']=get_ytm(price_dict['qh_buy_p'],price_dict['xh_sale_p'],price_dict['xh_now_p'])
price_dict['fantao']=get_ytm(price_dict['qh_sale_p'],price_dict['xh_buy_p'],price_dict['xh_now_p'])
price_dict['puretao']=get_ytm(price_dict['qh_now_p'],price_dict['xh_now_p'],price_dict['xh_now_p'])

price_dict['zhengtao_bp']=price_dict['qh_buy_p']-price_dict['xh_sale_p']
price_dict['fantao_bp']=price_dict['qh_sale_p']-price_dict['xh_buy_p']
price_dict['puretao_bp']=price_dict['qh_now_p']-price_dict['xh_now_p']


for i,k in price_dict.items():
    price_dict[i]=round(float(k),2)
return price_dict


{'xh_sale_p': 390.86,
 'xh_buy_p': 390.75,
 'xh_now_p': 390.88,
 'qh_sale_p': 396.5,
 'qh_buy_p': 396.46,
 'qh_now_p': 396.48,
 'zhengtao': 3.09,
 'fantao': 3.18,
 'puretao': 3.09}

In [58]:
# 获取期货合约列表
url='http://vip.stock.finance.sina.com.cn/quotes_service/api/json_v2.php/Market_Center.getHQFuturesData?page=1&num=40&sort=symbol&asc=1&node=hj_qh&_s_r_a=init'
res=requests.get(url)


In [59]:
import  re
df=pd.DataFrame()
info_list=[]
for one in re.findall('{.*?}',res.text):
    one_list=(dict(eval(one))).values()
    info_list.append(one_list)
df=pd.DataFrame(info_list,columns=dict(eval(one)).keys())
df

,symbol,exchange,name,trade,settlement,presettlement,open,high,low,close,...,askvol1,volume,position,ticktime,tradedate,preclose,changepercent,bid,ask,prevsettlement
0,AU0,shfe,黄金连续,393.56,0.00,395.620,394.16,395.28,392.14,0.00,...,2,104360,40363,14:06:31,2020-12-30,393.860,-0.0052070,0.00,0.00,395.62
1,AU2101,shfe,黄金2101,392.94,0.00,392.820,391.20,395.96,391.00,0.00,...,1,40,21,14:06:08,2020-12-30,391.020,0.0003055,0.00,0.00,392.82
2,AU2102,shfe,黄金2102,393.56,0.00,395.620,394.16,395.28,392.14,0.00,...,2,104360,40363,14:06:31,2020-12-30,393.860,-0.0052070,0.00,0.00,395.62
3,AU2103,shfe,黄金2103,395.50,0.00,396.700,394.58,395.50,394.06,0.00,...,1,4,28,14:06:05,2020-12-30,394.540,-0.0030250,0.00,0.00,396.70
4,AU2104,shfe,黄金2104,394.18,0.00,396.180,394.58,395.76,392.74,0.00,...,13,24900,37260,14:06:31,2020-12-30,394.560,-0.0050482,0.00,0.00,396.18
5,AU2106,shfe,黄金2106,395.90,0.00,397.860,395.74,397.54,394.40,0.00,...,2,38225,88126,14:06:31,2020-12-30,396.180,-0.0049264,0.00,0.00,397.86
6,AU2108,shfe,黄金2108,397.68,0.00,399.880,397.98,399.40,396.46,0.00,...,4,7700,21437,14:06:31,2020-12-30,397.980,-0.0055017,0.00,0.00,399.88
7,AU2110,shfe,黄金连续,399.92,0.00,401.760,399.88,401.08,398.48,0.00,...,7,682,1614,14:06:31,2020-12-30,399.820,-0.0045798,0.00,0.00,401.76
8,AU2112,shfe,黄金2112,401.80,0.00,403.440,401.42,402.56,401.04,0.00,...,1,11,114,14:04:57,2020-12-30,401.900,-0.0040650,0.00,0.00,403.44


In [53]:
df.columns.tolist()

['symbol',
 'exchange',
 'name',
 'trade',
 'settlement',
 'presettlement',
 'open',
 'high',
 'low',
 'close',
 'bidprice1',
 'askprice1',
 'bidvol1',
 'askvol1',
 'volume',
 'position',
 'ticktime',
 'tradedate',
 'preclose',
 'changepercent',
 'bid',
 'ask',
 'prevsettlement']

In [103]:
df_s=df[['symbol','trade','bidprice1','askprice1']].iloc[1:]
df_s
print(df_s.dtypes)

symbol       object
trade        object
bidprice1    object
askprice1    object
dtype: object


In [51]:
buy_sale_price,current_price

(['卖一', '390.24', '1', '买一', '390.14', '1'], '390.13')

In [62]:
xh_now=current_price
xh_buy=buy_sale_price[4]
xh_sale=buy_sale_price[1]


def get_ytm(buy,sale,now,date='2021-6-16'):
    now=datetime.
    delivery_day = datetime.datetime.strptime(date, '%Y-%m-%d') #到期日
    today = datetime.datetime.today()
    day_to_delivery = (delivery_day - today).days + 1  # 补上半天的差

    ytm=(buy-sale)* 365 / ((day_to_delivery) * now)*100
    return ytm


In [70]:
au_jys_text=requests.get('http://www.shfe.com.cn/products/au/').text


In [121]:
soup=BeautifulSoup(au_jys_text,'html5lib')
au_hy_list=soup.find_all('table',class_='listshuju')[0].find_all('tr')
info_list=[]
for one in au_hy_list:
    info_list.append(re.split('[\n\t ]+',one.text.strip()))
delivery_info=pd.DataFrame(info_list,columns=['symbol','listed','due','delivery_start','delivery_end','price'])
delivery_info['symbol']=delivery_info['symbol'].apply(lambda x:x.upper())
delivery_info.sort_values('symbol',inplace=True)
delivery_info
df_s['delivery']=delivery_info.delivery_end.tolist()

In [122]:
for one in df_s. columns.tolist()[1:]:
    df_s[one]=df_s[one].astype(float)
df_s

,symbol,trade,bidprice1,askprice1,jiacha0,jiacha_fan,jiacha_zheng,delivery
1,AU2101,392.94,392.50,393.76,2.81,3.62,2.26,20210118.0
2,AU2102,393.56,393.54,393.56,3.43,3.42,3.30,20210208.0
3,AU2103,395.50,393.84,394.98,5.37,4.84,3.60,20210316.0
4,AU2104,394.18,394.12,394.20,4.05,4.06,3.88,20210416.0
5,AU2106,395.90,395.90,395.92,5.77,5.78,5.66,20210616.0
6,AU2108,397.68,397.62,397.70,7.55,7.56,7.38,20210817.0
7,AU2110,399.92,399.62,399.74,9.79,9.60,9.38,20211018.0
8,AU2112,401.80,401.20,401.74,11.67,11.60,10.96,20211216.0


symbol        object
trade        float64
bidprice1    float64
askprice1    float64
dtype: object

In [120]:
df_s['jiacha0']=df_s.trade-float(xh_now)
df_s['jiacha_fan']=df_s.askprice1-float(xh_buy)
df_s['jiacha_zheng']=df_s.bidprice1-float(xh_sale)
df_s

,symbol,trade,bidprice1,askprice1,jiacha0,jiacha_fan,jiacha_zheng
1,AU2101,392.94,392.50,393.76,2.81,3.62,2.26
2,AU2102,393.56,393.54,393.56,3.43,3.42,3.30
3,AU2103,395.50,393.84,394.98,5.37,4.84,3.60
4,AU2104,394.18,394.12,394.20,4.05,4.06,3.88
5,AU2106,395.90,395.90,395.92,5.77,5.78,5.66
6,AU2108,397.68,397.62,397.70,7.55,7.56,7.38
7,AU2110,399.92,399.62,399.74,9.79,9.60,9.38
8,AU2112,401.80,401.20,401.74,11.67,11.60,10.96


In [157]:
df_s.dtypes.tolist()[1]==float

True

In [150]:
dict({'symbol':df.symbol.tolist()})

{'symbol': ['AU0',
  'AU2101',
  'AU2102',
  'AU2103',
  'AU2104',
  'AU2106',
  'AU2108',
  'AU2110',
  'AU2112']}

In [133]:
df_s['bidprice1']*df_s['askprice1']

1    154550.8000
2    154881.6024
3    155558.9232
4    155362.1040
5    156744.7280
6    158133.4740
7    159744.0988
8    161178.0880
dtype: float64

In [117]:
import datetime
datetime.datetime.strptime('20201212', '%Y%m%d') #到期日


datetime.datetime(2020, 12, 12, 0, 0)

In [145]:
def get_ytm(jiacha,xh_now,date):

    delivery_day = date.apply(lambda date:datetime.datetime.strptime(str(int(date)), '%Y%m%d')) #到期日
    today = datetime.datetime.today()
    day_to_delivery = (delivery_day.apply(lambda x:(x-today).days+1))  # 补上半天的差

#     print(jiacha,day_to_delivery*xh_now)
    ytm=jiacha* 365 / ((day_to_delivery) * float(xh_now))*100
    return round(ytm,2)

In [146]:
df_s['puretao']=get_ytm(jiacha=df_s['jiacha0'],xh_now=xh_now,date=df_s['delivery'])
df_s['fantao']=get_ytm(jiacha=df_s['jiacha_fan'],xh_now=xh_now,date=df_s['delivery'])
df_s['zhengtao']=get_ytm(jiacha=df_s['jiacha_zheng'],xh_now=xh_now,date=df_s['delivery'])

df_s

,symbol,trade,bidprice1,askprice1,jiacha0,jiacha_fan,jiacha_zheng,delivery,ytm
1,AU2101,392.94,392.50,393.76,2.81,3.62,2.26,20210118.0,13.84
2,AU2102,393.56,393.54,393.56,3.43,3.42,3.30,20210208.0,8.02
3,AU2103,395.50,393.84,394.98,5.37,4.84,3.60,20210316.0,6.61
4,AU2104,394.18,394.12,394.20,4.05,4.06,3.88,20210416.0,3.54
5,AU2106,395.90,395.90,395.92,5.77,5.78,5.66,20210616.0,3.21
6,AU2108,397.68,397.62,397.70,7.55,7.56,7.38,20210817.0,3.07
7,AU2110,399.92,399.62,399.74,9.79,9.60,9.38,20211018.0,3.14
8,AU2112,401.80,401.20,401.74,11.67,11.60,10.96,20211216.0,3.11


array([['AU2101', 392.94, 392.5, 393.76, 2.8100000000000023,
        3.6200000000000045, 2.259999999999991, 20210118.0, 13.84],
       ['AU2102', 393.56, 393.54, 393.56, 3.430000000000007,
        3.420000000000016, 3.3000000000000114, 20210208.0, 8.02],
       ['AU2103', 395.5, 393.84, 394.98, 5.3700000000000045,
        4.840000000000032, 3.599999999999966, 20210316.0, 6.61],
       ['AU2104', 394.18, 394.12, 394.2, 4.050000000000011,
        4.060000000000002, 3.8799999999999955, 20210416.0, 3.54],
       ['AU2106', 395.9, 395.9, 395.92, 5.769999999999982,
        5.78000000000003, 5.659999999999968, 20210616.0, 3.21],
       ['AU2108', 397.68, 397.62, 397.7, 7.550000000000011,
        7.560000000000002, 7.3799999999999955, 20210817.0, 3.07],
       ['AU2110', 399.92, 399.62, 399.74, 9.79000000000002,
        9.600000000000023, 9.379999999999995, 20211018.0, 3.14],
       ['AU2112', 401.8, 401.2, 401.74, 11.670000000000016,
        11.600000000000023, 10.95999999999998, 20211216.0, 

In [185]:
def contract_list_sina(): # sina,可以直接获取全合约，不用指定合约代码，带有标签，所以数据量会大些，获取时间更长,AU2110标签有误
    global qh_symbol_list
    url = 'http://vip.stock.finance.sina.com.cn/quotes_service/api/json_v2.php/' \
          'Market_Center.getHQFuturesData?page=1&num=40&sort=symbol&asc=1&node=hj_qh&_s_r_a=init'
    res = requests.get(url)
    info_list = []
    for one in re.findall('{.*?}', res.text):
        one_list = (dict(eval(one))).values()
        info_list.append(one_list)
    df = pd.DataFrame(info_list, columns=list(dict(eval(one)).keys()))
    df.set_index('symbol',inplace=True)
    need_info=['volume','position','tradedate']# 交易量，成交量
    # 因为为合约信息，删掉部分内容
    df=df[need_info]
    return df


# 合约信息包括部分行情最好在收市后更新一次，大约四点？
def contract_list(): # 上海期货交易所数据，可以直接获取全合约，不用指定合约代码
    
    from bs4 import BeautifulSoup
    au_jys_text = requests.get('http://www.shfe.com.cn/products/au/').text
    soup = BeautifulSoup(au_jys_text, 'html5lib')
    au_hy_list = soup.find_all('table', class_='listshuju')[0].find_all('tr')
    info_list = []
    for one in au_hy_list:
        info_list.append(re.split('[\n\t ]+', one.text.strip()))
    delivery_info = pd.DataFrame(info_list,
                                 columns=['symbol', 'listed_day', 'due', 'delivery_start', 'delivery_day', 'price'])
    delivery_info['symbol'] = delivery_info['symbol'].apply(lambda x: x.upper())# 取大写
    delivery_info.sort_values('symbol', inplace=True)# 按代码排序
    
    delivery_info=delivery_info[['symbol','listed_day','delivery_day']]# 代码，上市日，到期日
    delivery_info.set_index('symbol',inplace=True)
    contract_other_info=contract_list_sina().loc[delivery_info.index.tolist()]
    delivery_info=pd.concat([delivery_info,contract_other_info],axis=1)
    return delivery_info

In [186]:

delivery_info=contract_list()
delivery_info.

,listed_day,delivery_day,volume,position,tradedate
symbol,,,,,
AU2101,20201016,20210118,45,19,2020-12-30
AU2102,20200116,20210208,35167,38494,2020-12-31
AU2103,20201216,20210316,1,29,2020-12-31
AU2104,20200317,20210416,8807,37720,2020-12-31
AU2106,20200518,20210616,12763,88831,2020-12-31
AU2108,20200716,20210817,2262,21836,2020-12-31
AU2110,20200916,20211018,72,1614,2020-12-31
AU2112,20201117,20211216,5,117,2020-12-31


In [178]:
contract_list_sina().loc[delivery_info.index.tolist()].T
need_info=['volume','postion','traddate']# 交易量，成交量

symbol,AU2101,AU2102,AU2103,AU2104,AU2106,AU2108,AU2110,AU2112
exchange,shfe,shfe,shfe,shfe,shfe,shfe,shfe,shfe
name,黄金2101,黄金2102,黄金2103,黄金2104,黄金2106,黄金2108,黄金连续,黄金2112
trade,392.38,394.28,393.94,394.96,396.70,398.12,400.08,402.04
settlement,393.26,0.00,0.00,0.00,0.00,0.00,0.00,0.00
presettlement,392.820,393.940,394.780,394.600,396.200,398.140,400.220,401.680
open,391.20,393.40,393.94,393.86,395.70,397.54,400.26,401.10
high,395.96,394.94,393.94,395.58,397.34,398.92,400.68,403.76
low,391.00,392.64,393.94,393.40,395.10,396.82,399.84,401.10
close,392.38,0.00,0.00,0.00,0.00,0.00,0.00,0.00
bidprice1,390.000,394.260,394.740,394.920,396.680,398.160,400.040,401.900


In [191]:
def au_real_time():
    """
    黄金交易所数据
    :return:
    """
    url = 'https://www.sge.com.cn/graph/quotations'
    form_data = {'instid': 'Au(T+D)'}
    res = requests.post(url, data=form_data)
    au_td = pd.DataFrame(json.loads(res.text))[['times', 'data']].set_index('times')
    au_td = au_td.astype(float)
    au_td.replace(au_td.iloc[-1], np.nan, inplace=True)
    au_td.dropna(inplace=True)
    return au_td


def one_future_real_time(symbol='AU2106'):
    # 改为t5可获取5日的
    url = 'https://stock2.finance.sina.com.cn/futures/api/jsonp.php/' \
          'var%20t1nf_AU2106=/InnerFuturesNewService.getMinLine?symbol={}'.format(symbol)
    res = requests.get(url)
    df = pd.DataFrame(eval(res.text[res.text.index('('):][1:-2]), columns=[
        'times', 'price', 'avg_price', 'deal_amount', 'open_interest', 'info', 'info'
    ]).set_index('times').iloc[:, :-2]
    df.price = df.price.astype('float')
    return pd.DataFrame(df.price)


def update_hist(au_and_future):
    close_data = au_and_future.loc['15:00'].tolist()
    date = str(datetime.datetime.now())[:10]
    with open('./data/Au/0_close_hist.csv', 'a') as f:
        f.write(','.join([date] + [str(x) for x in close_data]) + '\n')
        f.close()


def get_both():
    future_oneday_real_time = one_future_real_time(symbol='AU2106')
    au_oneday_real_time = au_real_time()
    au_oneday_real_time = au_oneday_real_time[~au_oneday_real_time.index.duplicated(keep='first')]  # 因为可能有重复值，去重
    au_and_future = pd.concat([future_oneday_real_time, au_oneday_real_time], axis=1)
    au_and_future.columns = ['future', 'Au_TD']
    # 排序
    night_price = au_and_future.sort_index()['17:00':]
    day_price = au_and_future.sort_index()[:'17:00']
    au_and_future = pd.concat([night_price, day_price], axis=0)
    # 计算数据
    au_and_future['diff'] = au_and_future['future'] - au_and_future['Au_TD']
    delivery_day = datetime.datetime.strptime('2021-6-16', '%Y-%m-%d')  # 到期日
    today = datetime.datetime.today()
    day_to_delivery = (delivery_day - today).days + 1  # 补上半天的差
    au_and_future['ytm'] = au_and_future['diff'] * 365 / ((day_to_delivery) * au_and_future['Au_TD']) * 100
#     if str(datetime.datetime.now().time()) > '15:32':
#         day_str = str(datetime.datetime.now())[:10] + '.csv'
# #         if day_str not in os.listdir('./data/Au/'):
# #             au_and_future.to_csv('./data/Au/{}'.format(day_str))
# #             update_hist(au_and_future)
    return au_and_future

In [192]:
import os
get_both()

,future,Au_TD,diff,ytm
20:00,NaN,389.99,NaN,NaN
20:01,NaN,390.00,NaN,NaN
20:02,NaN,389.97,NaN,NaN
20:03,NaN,389.93,NaN,NaN
20:04,NaN,390.03,NaN,NaN
...,...,...,...,...
23:27,397.10,391.30,5.80,3.220340
23:28,397.14,391.27,5.87,3.259456
23:29,397.02,391.15,5.87,3.260456
23:30,397.00,NaN,NaN,NaN


In [230]:
td_high_freq_url='http://futsse.eastmoney.com/list/variety/118/0?orderBy=name&sort=desc&pageSize=12&pageIndex=0&callbackName=&cb=hh&_={time}'
res=requests.get(td_high_freq_url.format(time=int(time.time()*1000)))
res

<Response [200]>

In [233]:
res_dict=dict(eval(res.text[1:-1]))
for one in res_dict['list']:
    if  (one['name']=='黄金T+D'):
        '''
        {'qrspj': 390.05,
         'spsj': 1609399800,
         'np': 2824,
         'rz': 870,
         'dm': 'AUTD',
         'zsjd': 2,
         'lx': 0,
         'ccl': 191972,
         'ly': 'c12',
         'kpsj': 1609329600,
         'dt': 347.5,
         'sc': 118,
         'uid': 'SGE|AUTD',
         'vol': 6058,
         'bpgs': 1,
         'jysj': 827,
         'mcj': 391.3,
         'cjbs': 0,
         'mcl': 6,
         'wp': 3234,
         'cje': 2367491024,
         'mrj': 391.26,
         'utime': 1609344507,
         'jjsj': '-',
         'mrl': 1,
         'h': 391.88,
         'j': 390.8,
         'zccl': 191102,
         'l': 389.38,
         'zf': '-',
         'mmpl': [0, 0, 0, 0, 6, 1, 0, 0, 0, 0],
         'o': 390.0,
         'p': 391.25,
         'cclbh': 20,
         'xsfx': 2,
         'lb': '-',
         'name': '黄金T+D',
         'zde': 0.79,
         'zt': 433.41,
         'jyzt': 0,
         'xs': 20,
         'spgs': 1,
         'zdf': 0.2,
         'mmpjg': [0.0, 0.0, 0.0, 0.0, 391.3, 391.26, 0.0, 0.0, 0.0, 0.0],
         'zjsj': 390.46}'''
        
        bid_price=one['mrj']
        ask_price=one['mcj']
        now_price=one['p']
        break

In [234]:
one

{'qrspj': 390.05,
 'spsj': 1609399800,
 'np': 2824,
 'rz': 870,
 'dm': 'AUTD',
 'zsjd': 2,
 'lx': 0,
 'ccl': 191972,
 'ly': 'c12',
 'kpsj': 1609329600,
 'dt': 347.5,
 'sc': 118,
 'uid': 'SGE|AUTD',
 'vol': 6058,
 'bpgs': 1,
 'jysj': 827,
 'mcj': 391.3,
 'cjbs': 0,
 'mcl': 6,
 'wp': 3234,
 'cje': 2367491024,
 'mrj': 391.26,
 'utime': 1609344507,
 'jjsj': '-',
 'mrl': 1,
 'h': 391.88,
 'j': 390.8,
 'zccl': 191102,
 'l': 389.38,
 'zf': '-',
 'mmpl': [0, 0, 0, 0, 6, 1, 0, 0, 0, 0],
 'o': 390.0,
 'p': 391.25,
 'cclbh': 20,
 'xsfx': 2,
 'lb': '-',
 'name': '黄金T+D',
 'zde': 0.79,
 'zt': 433.41,
 'jyzt': 0,
 'xs': 20,
 'spgs': 1,
 'zdf': 0.2,
 'mmpjg': [0.0, 0.0, 0.0, 0.0, 391.3, 391.26, 0.0, 0.0, 0.0, 0.0],
 'zjsj': 390.46}

In [246]:
dict(pd.read_csv('./2020-12-31.csv',encoding='gbk',index_col=0).index)


ValueError: dictionary update sequence element #0 has length 6; 2 is required

In [241]:
str(datetime.datetime.now().time())[:8]

'00:52:53'

In [251]:
full_df=pd.DataFrame(eval(open('2020-12-31.txt').readline(-1)))
full_df

,name,buy,sale,current_price,delivery,jiacha0,jiacha_fan,jiacha_zheng,puretao,fantao,zhengtao,time
AU2101,黄金2101,392.64,395.54,0.00,20210118,-391.46,4.24,1.19,-2027.78,21.96,6.16,00:53:19
AU2102,黄金2102,394.84,394.88,394.88,20210208,3.42,3.58,3.39,8.18,8.56,8.10,00:53:19
AU2103,黄金2103,395.24,395.82,395.34,20210316,3.88,4.52,3.79,4.82,5.62,4.71,00:53:19
AU2104,黄金2104,395.46,395.52,395.52,20210416,4.06,4.22,4.01,3.57,3.71,3.53,00:53:19
AU2106,黄金2106,397.26,397.28,397.28,20210616,5.82,5.98,5.81,3.25,3.34,3.24,00:53:19
AU2108,黄金2108,398.76,398.86,398.82,20210817,7.36,7.56,7.31,3.00,3.08,2.98,00:53:19
AU2110,黄金2110,400.80,400.92,400.86,20211018,9.40,9.62,9.35,3.01,3.08,3.00,00:53:19
AU2112,黄金2112,401.98,403.74,402.78,20211216,11.32,12.44,10.53,3.02,3.31,2.81,00:53:19
AUTD,黄金TD,391.30,391.45,391.46,0,0.00,0.00,0.00,0.00,0.00,0.00,00:53:19


In [266]:
tot_dict=dict({k:v for k,v in zip(['xh_buy_p','xh_sale_p','xh_now_p'],full_df.loc['AUTD'][1:4].tolist())})
tot_dict

{'xh_buy_p': 391.3, 'xh_sale_p': 391.45, 'xh_now_p': 391.46}

AttributeError: 'dict' object has no attribute 'reverse'